# Notebook#2
-----
Este notebook tiene el objetivo de asignar una prioridad a cada documento de la familia de patentes del documento peruano examinado.

**Contexto**

Los documentos de patentes peruanos no cuentan con una publicación de su documento completo en las bases de datos de patentes, por ello para acceder a las reivindicaciones ha sido necesario recopilar otras patentes que pertenecen a la misma familia. Especificamente se han considerado solo las patentes publicadas en la oficina mexicana, española y en caso de no existir ninguna de estas, se ha considerado la publicación internacional.

## 1. Leer los resultados de Patstat

Los documentos técnicos de las patentes evaluadas en las resoluciones del Indecopi fueron obtenidos a través de la familia de patentes publicadas en las oficinas de México y España. En los casos que no se hallaron dicha spublicaicones se descargó la publicación internacional en caso exista.

> *La [query](https://github.com/josemza/patent_infringement_486/blob/main/sql/query_patstat.sql) utilizada para extraer esta información se encuentra en el siguiente directorio del proyecto:*
>
> ```
> patent_infringement_486/
> └──sql/
>     └──query_patstat.py
> ```

In [1]:
import pandas as pd

df_family = pd.read_csv('patentes_familia_MX_ES_WO_art15.csv', sep=';')
df_family.head()

,numsol,publication,appln_title,applicants,fam_publn_auth,fam_publn_nr,fam_publn_kind
0,PE2004000641A,PE20050405A1,"COMPOSICION A BASE DE LIPIDOS MICROBIANOS, PAR...",NORFERM DA|NORFERM DA|NORFERM DA|NORFERM DA|NO...,WO,2005004888,A1
1,PE2005000158A,PE20060032A1,"METODO, SISTEMA Y PROTOCOLO PARA RETROALIMENTA...",Nokia Corporation|Nokia Corporation|Nokia Corp...,WO,2005088931,A1
2,PE2005000159A,PE20051168A1,BENZONITRILOS DERIVADOS COMO MODULADORES DE RE...,Warner-Lambert Company LLC|Warner-Lambert Comp...,WO,2005080320,A1
3,PE2005000780A,PE20060377A1,PROCEDIMIENTO DE PREPARACION DE UNA VACUNA BIV...,INSTITUTO NACIONAL DE PSIQUIATRIA RAMON DE LA ...,ES,2435769,T3
4,PE2005000780A,PE20060377A1,PROCEDIMIENTO DE PREPARACION DE UNA VACUNA BIV...,INSTITUTO NACIONAL DE PSIQUIATRIA RAMON DE LA ...,MX,PA04006617,A


Concatenar con "/" el **código de la oficina**, el **número de publicación** y el ***kind id*** de la publicación
> *- Este formato de codificación del número de publicaicón de la patente sirve para construir la estructura de peticion aceptada por la API de la OPS.*
>> Ejemplo de *request*
http://ops.epo.org/rest-services/published-data/images/ES/2843204/T3/fullimage.tiff?Range=1
>
> *- Para mayor detalle sobre la forma de uso revisar el código del archivo [ops_download.py](https://github.com/josemza/patent_infringement_486/blob/main/scripts/ops_download.py)*
>
> *- Puedes revisar la documentación de la API [aquí](https://link.epo.org/web/searching-for-patents/data/en-ops-v3.2-documentation-version-1.3.20.pdf)*

## 2. Procesar datos

In [2]:
cols = ['fam_publn_auth', 'fam_publn_nr', 'fam_publn_kind']

df_family['family_doc_final'] = (
    df_family[cols]
      .astype(str)
      .agg('/'.join, axis=1)
)

df_family.head()

,numsol,publication,appln_title,applicants,fam_publn_auth,fam_publn_nr,fam_publn_kind,family_doc_final
0,PE2004000641A,PE20050405A1,"COMPOSICION A BASE DE LIPIDOS MICROBIANOS, PAR...",NORFERM DA|NORFERM DA|NORFERM DA|NORFERM DA|NO...,WO,2005004888,A1,WO/2005004888/A1
1,PE2005000158A,PE20060032A1,"METODO, SISTEMA Y PROTOCOLO PARA RETROALIMENTA...",Nokia Corporation|Nokia Corporation|Nokia Corp...,WO,2005088931,A1,WO/2005088931/A1
2,PE2005000159A,PE20051168A1,BENZONITRILOS DERIVADOS COMO MODULADORES DE RE...,Warner-Lambert Company LLC|Warner-Lambert Comp...,WO,2005080320,A1,WO/2005080320/A1
3,PE2005000780A,PE20060377A1,PROCEDIMIENTO DE PREPARACION DE UNA VACUNA BIV...,INSTITUTO NACIONAL DE PSIQUIATRIA RAMON DE LA ...,ES,2435769,T3,ES/2435769/T3
4,PE2005000780A,PE20060377A1,PROCEDIMIENTO DE PREPARACION DE UNA VACUNA BIV...,INSTITUTO NACIONAL DE PSIQUIATRIA RAMON DE LA ...,MX,PA04006617,A,MX/PA04006617/A


Asignar prioridad al documento según la oficina de publicación:

- (1) España
- (2) México
- (3) WO - publicaicón internacional

In [3]:
df_family['prior'] = df_family['fam_publn_auth'].apply(lambda x: 1 if x == 'ES' else 2 if x == 'MX' else 3)
df_family.head()

,numsol,publication,appln_title,applicants,fam_publn_auth,fam_publn_nr,fam_publn_kind,family_doc_final,prior
0,PE2004000641A,PE20050405A1,"COMPOSICION A BASE DE LIPIDOS MICROBIANOS, PAR...",NORFERM DA|NORFERM DA|NORFERM DA|NORFERM DA|NO...,WO,2005004888,A1,WO/2005004888/A1,3
1,PE2005000158A,PE20060032A1,"METODO, SISTEMA Y PROTOCOLO PARA RETROALIMENTA...",Nokia Corporation|Nokia Corporation|Nokia Corp...,WO,2005088931,A1,WO/2005088931/A1,3
2,PE2005000159A,PE20051168A1,BENZONITRILOS DERIVADOS COMO MODULADORES DE RE...,Warner-Lambert Company LLC|Warner-Lambert Comp...,WO,2005080320,A1,WO/2005080320/A1,3
3,PE2005000780A,PE20060377A1,PROCEDIMIENTO DE PREPARACION DE UNA VACUNA BIV...,INSTITUTO NACIONAL DE PSIQUIATRIA RAMON DE LA ...,ES,2435769,T3,ES/2435769/T3,1
4,PE2005000780A,PE20060377A1,PROCEDIMIENTO DE PREPARACION DE UNA VACUNA BIV...,INSTITUTO NACIONAL DE PSIQUIATRIA RAMON DE LA ...,MX,PA04006617,A,MX/PA04006617/A,2


## 3. Exportar

In [ ]:
df_family['family_doc_final'].to_csv(
                                'patentes.csv', 
                                sep=',', 
                                index=False, 
                                header=['doc_number']
                            )